In [59]:
import pandas as pd

akas_df = pd.read_csv('data/title.akas.tsv', sep='\t', dtype=str, na_values=["\\N"])

title_df = pd.read_csv('data/title.basics.tsv', sep='\t', dtype=str, na_values=["\\N"])

crew_df = pd.read_csv("data/title.crew.tsv", sep="\t", dtype=str, na_values=["\\N"])

principals_df = pd.read_csv('data/title.principals.tsv', sep='\t', dtype=str, na_values=["\\N"])

ratings_df = pd.read_csv("data/title.ratings.tsv", sep="\t", dtype=str, na_values=["\\N"])

names_df = pd.read_csv("data/name.basics.tsv", sep="\t", dtype=str, na_values=["\\N"])

In [60]:
akas_df = akas_df.loc[akas_df['region'] == 'US']

akas_df = akas_df.loc[akas_df['types'] != 'tv']

akas_df.drop(columns=['ordering', 'title', 'attributes', 'isOriginalTitle', 'language', 'types', 'region'], inplace=True)

In [62]:
akas_df.to_csv("data/all.csv", index=False, sep=";")

In [61]:
title_df.drop(columns=['originalTitle', 'isAdult', 'endYear'], inplace=True)

title_df = title_df.loc[title_df['titleType'] == 'movie']

title_df.drop(columns=['titleType'], inplace=True)

In [63]:
title_df.to_csv("data/movies.csv", index=False, sep=";")

In [68]:
crew_df.drop(columns=['writers'], inplace=True)

KeyError: "['writers'] not found in axis"

In [46]:
principals_df.loc[principals_df['category'] == 'actor']

principals_df.drop(columns=['ordering', 'category', 'job', 'characters'], inplace=True)

,tconst,nconst
0,tt0000001,nm1588970
1,tt0000001,nm0005690
2,tt0000001,nm0005690
3,tt0000001,nm0374658
4,tt0000002,nm0721526


In [65]:
ratings_df.drop(columns=['numVotes'], inplace=True)

In [66]:
names_df.drop(columns=['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [67]:
title_df.to_csv("data/data.csv", index=False, sep=";")

In [70]:
data = pd.merge(akas_df, title_df, left_on='titleId', right_on='tconst', how='outer')

data = pd.merge(data, crew_df, on='tconst', how='outer')

data = pd.merge(data, ratings_df, on='tconst', how='outer')

data = pd.merge(data, names_df, left_on='directors', right_on='nconst', how='outer')

data['directors'] = data['primaryName']

data.drop(columns=['tconst', 'nconst', 'primaryName'], inplace=True)

data.head()

,titleId,primaryTitle,startYear,runtimeMinutes,genres,directors,averageRating
0,NaN,NaN,NaN,NaN,NaN,Fred Astaire,NaN
1,NaN,NaN,NaN,NaN,NaN,Lauren Bacall,NaN
2,NaN,NaN,NaN,NaN,NaN,Brigitte Bardot,NaN
3,NaN,NaN,NaN,NaN,NaN,John Belushi,NaN
4,tt0038468,It Rains on Our Love,1946,95,"Drama,Romance",Ingmar Bergman,6.6


In [71]:
data.to_csv("data/data.csv", index=False, sep=";")